# NLI base results: Mistral-Nemo-Instruct-2407 (mistralai/Mistral-Nemo-Instruct-2407)

Loads [yilmazzey/sdp2-nli](https://huggingface.co/datasets/yilmazzey/sdp2-nli) (snli_tr_1_1, multinli_tr_1_1, trglue_mnli) and runs **test-only** evaluation with this model.

12B generative LLM (Mistral-Nemo series, instruct-tuned). Multilingual support including Turkish. Zero-shot prompted NLI (no fine-tuning). Expected strong performance on reasoning/NLI tasks. Outputs parsed to 0=entailment, 1=neutral, 2=contradiction.

**Splits:** snli → test; multinli → validation_matched/mismatched; trglue → test_matched/test_mismatched. **Metrics:** Accuracy, macro F1, per-class F1, confusion matrix (CSV + plot).

In [2]:
# Colab: uncomment and run once to install/upgrade (Runtime -> Change runtime type -> GPU)
!pip install -q -U transformers datasets accelerate scikit-learn tqdm

import json
import random
from collections import Counter
from pathlib import Path

import numpy as np
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_PLOT = True
except ImportError:
    HAS_PLOT = False

LABEL_NAMES = ["entailment", "neutral", "contradiction"]

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU; running on CPU (12B requires GPU for practical use).")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No GPU; running on CPU (12B requires GPU for practical use).


In [3]:
REPO_ID = "yilmazzey/sdp2-nli"
CONFIGS = ["snli_tr_1_1", "multinli_tr_1_1", "trglue_mnli"]
MODEL_ID = "mistralai/Mistral-Nemo-Instruct-2407"
NUM_LABELS = 3  # entailment, neutral, contradiction
RESULTS_DIR = "results"
# Lower to 4-8 if memory low (12B model requires GPU). Adjust based on available VRAM.
BATCH_SIZE = 4
EVAL_SPLITS = {
    "snli_tr_1_1": ["test"],
    "multinli_tr_1_1": ["validation_matched", "validation_mismatched"],
    "trglue_mnli": ["test_matched", "test_mismatched"],
}

In [4]:
# Load all three dataset configs
datasets = {}
for cfg in CONFIGS:
    print(f"Loading {REPO_ID} :: {cfg} ...")
    datasets[cfg] = load_dataset(REPO_ID, cfg)
    print("  splits:", list(datasets[cfg].keys()))

Loading yilmazzey/sdp2-nli :: snli_tr_1_1 ...
  splits: ['train', 'validation', 'test']
Loading yilmazzey/sdp2-nli :: multinli_tr_1_1 ...
  splits: ['train', 'validation_matched', 'validation_mismatched']
Loading yilmazzey/sdp2-nli :: trglue_mnli ...
  splits: ['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched']


In [5]:
print("Loading Mistral-Nemo-Instruct-2407 (text-generation pipeline)...")

generator = pipeline(
    "text-generation",
    model=MODEL_ID,
    device=0 if torch.cuda.is_available() else -1,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    max_length=None,  # Silence warning
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

if hasattr(tokenizer, "pad_token") and tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model loaded successfully.")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading Mistral-Nemo-Instruct-2407 (text-generation pipeline)...


Loading weights: 100%|██████████| 363/363 [00:42<00:00,  8.61it/s, Materializing param=model.norm.weight]                              
The tokenizer you are loading from 'mistralai/Mistral-Nemo-Instruct-2407' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Passing `generation_config` together with generation-related arguments=({'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The tokenizer you are loading from 'mistralai/Mistral-Nemo-Instruct-2407' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You s

Model loaded successfully.


In [6]:
def nli_prompt(premise, hypothesis):
    return [
        {"role": "system", "content": "You are a helpful assistant for natural language inference. Classify the relationship between premise and hypothesis as entailment, neutral, or contradiction. Respond with exactly one word: entailment, neutral, or contradiction. No explanation or additional text."},
        {"role": "user", "content": f"Premise: {premise}\nHypothesis: {hypothesis}\nLabel:"}
    ]

def parse_generated_label(gen_text, prompt_text):
    continuation = gen_text[len(prompt_text):].strip().lower()
    if not continuation:
        return 1  # neutral fallback
    first_word = continuation.split()[0].rstrip('.,!?')
    label_map = {"entailment": 0, "neutral": 1, "contradiction": 2}
    return label_map.get(first_word, 1)  # Default to neutral if unknown

def run_prompted_inference(ds):
    premises = list(ds["premise"])
    hypotheses = list(ds["hypothesis"])
    labels = list(ds["label"])
    n = len(ds)
    y_pred = []
    all_generations = []  # Collect all for full debug

    for start in tqdm(range(0, n, BATCH_SIZE), desc="Inference"):
        batch_premises = premises[start : start + BATCH_SIZE]
        batch_hypotheses = hypotheses[start : start + BATCH_SIZE]
        batch_prompts = [nli_prompt(p, h) for p, h in zip(batch_premises, batch_hypotheses)]

        formatted_prompts = tokenizer.apply_chat_template(batch_prompts, tokenize=False, add_generation_prompt=True)

        out = generator(
            formatted_prompts,
            max_new_tokens=5,  # Strict limit
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
            max_length=None,  # Silence warning
        )

        all_generations.extend(out)  # Save all

        for i, gen in enumerate(out):
            gen_text = gen[0]["generated_text"]
            y_pred.append(parse_generated_label(gen_text, formatted_prompts[i]))

    y_true = np.array(labels, dtype=np.int64)
    y_pred = np.array(y_pred, dtype=np.int64)

    # Log first 5 generations for debug
    for i in range(min(5, n)):
        print(f"Debug Sample {i}: Generated: {all_generations[i][0]['generated_text']}, Parsed Label: {y_pred[i]}")

    print("True label dist:", dict(Counter(y_true)))
    print("Pred label dist:", dict(Counter(y_pred)))

    return y_true, y_pred

In [7]:
def compute_metrics(y_true, y_pred):
    acc = float(accuracy_score(y_true, y_pred))
    f1_macro = float(f1_score(y_true, y_pred, average="macro", zero_division=0))
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = {LABEL_NAMES[i]: float(f1_per_class[i]) for i in range(NUM_LABELS)}
    cm = confusion_matrix(y_true, y_pred)
    out = {"accuracy": acc, "f1_macro": f1_macro, "f1_per_class": f1_per_class}
    return out, cm


def save_confusion_plot(cm, path):
    if not HAS_PLOT:
        return
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

In [ ]:
# Quick test on subset (100 examples per split) - Run this first to test!
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
all_metrics = {}

for config_name in CONFIGS:
    ds_dict = datasets[config_name]
    split_names = EVAL_SPLITS[config_name]
    all_metrics[config_name] = {}

    for split_name in split_names:
        if split_name not in ds_dict:
            print(f"  Skip {config_name}/{split_name} (missing)")
            continue
        # Take only first 100 examples for quick test
        full_ds = ds_dict[split_name]
        ds = full_ds.select(range(min(100, len(full_ds))))
        print(f"Evaluating {config_name} / {split_name} (subset: {len(ds)}/{len(full_ds)}) ...")
        y_true, y_pred = run_prompted_inference(ds)
        metrics, cm = compute_metrics(y_true, y_pred)
        all_metrics[config_name][split_name] = metrics

        cm_path = Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}_subset.csv"
        np.savetxt(cm_path, cm, fmt="%d", delimiter=",")
        save_confusion_plot(cm, Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}_subset.png")

        print(f"  accuracy={metrics['accuracy']:.4f}, f1_macro={metrics['f1_macro']:.4f}")

with open(Path(RESULTS_DIR) / "metrics_subset.json", "w") as f:
    json.dump(all_metrics, f, indent=2)
print(f"Saved {RESULTS_DIR}/metrics_subset.json")

Evaluating snli_tr_1_1 / test (subset: 100/9824) ...


Inference:  40%|████      | 10/25 [4:57:14<7:13:13, 1732.91s/it]

In [8]:
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
all_metrics = {}

for config_name in CONFIGS:
    ds_dict = datasets[config_name]
    split_names = EVAL_SPLITS[config_name]
    all_metrics[config_name] = {}

    for split_name in split_names:
        if split_name not in ds_dict:
            print(f"  Skip {config_name}/{split_name} (missing)")
            continue
        ds = ds_dict[split_name]
        print(f"Evaluating {config_name} / {split_name} ...")
        y_true, y_pred = run_prompted_inference(ds)
        metrics, cm = compute_metrics(y_true, y_pred)
        all_metrics[config_name][split_name] = metrics

        cm_path = Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}.csv"
        np.savetxt(cm_path, cm, fmt="%d", delimiter=",")
        save_confusion_plot(cm, Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}.png")

        print(f"  accuracy={metrics['accuracy']:.4f}, f1_macro={metrics['f1_macro']:.4f}")

with open(Path(RESULTS_DIR) / "metrics.json", "w") as f:
    json.dump(all_metrics, f, indent=2)
print(f"Saved {RESULTS_DIR}/metrics.json")

Evaluating snli_tr_1_1 / test ...


Inference:   0%|          | 0/2456 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'pad_token_id', 'do_sample', 'temperature', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Inference:   1%|          | 14/2456 [7:33:39<1318:51:36, 1944.27s/it]


KeyboardInterrupt: 

In [ ]:
# Summary: per config/split
for config_name, splits in all_metrics.items():
    for split_name, m in splits.items():
        print(f"{config_name} / {split_name}: acc={m['accuracy']:.4f}, F1_macro={m['f1_macro']:.4f}, F1_per_class={m['f1_per_class']}")